# Welcher Parlamentarier hat bei Google den grössten Fussabdruck?

1. Liste aller Parlamentarier finden (Pandas)
2. Liste mit Vornamen, Nachnamen und Rat bauen (Pandas)
3. Googlen und Resultate direkt ansteuern (Selenium)
4. DataFrame bauen und Resulate abspeichern (For Loop into Pandas)
5. Mergen wir die Daten mit dem ursprünglichen DataFrame
6. Visualisieren wir die Daten

## Imports

In [24]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import os
from bs4 import BeautifulSoup
from random import uniform

In [25]:
%matplotlib notebook
import matplotlib.pyplot as plt

**1. Liste aller Parlamentarier**

In [26]:
df = pd.read_excel('https://www.parlament.ch/Poly/Adressen_RM/mitgliederlist-pc.xlsx')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rat                     246 non-null    object 
 1   Anrede 1                246 non-null    object 
 2   Vorname                 246 non-null    object 
 3   Name                    246 non-null    object 
 4   Anrede 2                61 non-null     object 
 5   Firma (Postadresse)     56 non-null     object 
 6   Strasse (Postadresse)   81 non-null     object 
 7   Zusatz (Postadresse)    27 non-null     object 
 8   PLZ (Postadresse)       81 non-null     float64
 9   Ort (Postadresse)       81 non-null     object 
 10  Tel. (Postadresse)      38 non-null     object 
 11  Fax (Postadresse)       18 non-null     object 
 12  Homepage (Postadresse)  37 non-null     object 
 13  Firma (Wohnadresse)     5 non-null      object 
 14  Strasse (Wohnadresse)   179 non-null    ob

**2. Liste mit Vornamen, Nachnamen und Rat bauen**

In [28]:
def make(elem):
    if elem == 'NR':
        return 'Nationalrat'
    else:
        return 'Ständerat'
df["Rat"] = df["Rat"].apply(make)

In [29]:
#Die wichtigsten herauskopieren
df_n = df[['Anrede 1', 'Anrede 2', 'Vorname', 'Name']].copy()
df_n = df_n.fillna(value="")

In [30]:
df_n['Anrede'] = df_n["Anrede 1"] + df_n['Anrede 2']

In [31]:
def clean(elem):
    elem = elem.replace('Monsieur', "").replace('Herr ', "").replace('Frau ', "")
    elem = elem.replace('Madame', "").replace('Vizepräsident des ', "").replace('Nationalrates', "Nationalrat")
    return elem

df_n['Anrede'] = df_n['Anrede'].apply(clean)

In [32]:
df_n['Google Search'] ='"' + df_n['Anrede'] + " " + df_n['Vorname'] + " " + df_n['Name']+'"'

In [33]:
pol_list = list(df_n['Google Search'])

In [34]:
pol_list[:5]

['"Conseiller national Jean-Luc Addor"',
 '"Nationalrat Andreas Aebi"',
 '"Nationalrat Matthias Aebischer"',
 '"Nationalrat Thomas Aeschi"',
 '"Conseillère nationale Céline Amaudruz"']

**3. Googlen und Resultate direkt ansteuern**

In [127]:
#Wir starten den BRowser auf
driver = webdriver.Firefox()

In [128]:
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://www.google.ch')

In [129]:
#Wir benutzen den Webinspector
search = driver.find_element_by_name('q')

In [130]:
#send keys
search.send_keys(pol_list[0])

In [131]:
#button click
search.send_keys(Keys.RETURN)

In [132]:
#get results in text format
driver.find_element_by_id('result-stats').text

'About 2,390 results (0.48 seconds) '

In [133]:
driver.find_element_by_name('q').clear()

**4. DataFrame bauen und Resulate abspeichern**

In [ ]:
#VPN Network -> Google

In [166]:
#Wir starten den Browser auf
driver = webdriver.Firefox()
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://www.google.ch')

In [167]:
driver.find_element_by_name('q').clear()

In [168]:
#start the for loop
result_list = []
for elem in pol_list:
    
    search = driver.find_element_by_name('q')
    search.send_keys(elem)
    time.sleep(1)
    search.send_keys(Keys.RETURN)
    #wait
    time.sleep(1)
    #get and save results
    result = driver.find_element_by_id('result-stats').text
    minidict = {'Politician': elem,
                'Google Search Results': result}
    result_list.append(minidict)
    
    driver.find_element_by_name('q').clear()
    
    #wait
    time.sleep(uniform(1, 3))
    

In [169]:
#len(result_list)

35

**5. Abspeichern und alle Files zusammenbringen, um sie mit dem ursprünglichen DataFrame zu verbinden**

In [35]:
csvlist = os.listdir('csvs')

In [36]:
df_count = pd.DataFrame([])

In [37]:
for csv in csvlist:
    temp = pd.read_csv("csvs/"+csv)
    frames = [df_count, temp]
    df_count = pd.concat(frames)

In [44]:
df_count_merged = df_count.merge(df_n, left_on='Politician', right_on='Google Search')

In [48]:
#Namemerge
df_count_merged['mergeonname'] = df_count_merged['Vorname'] + df_count_merged['Name']
df['mergeonname'] = df['Vorname'] + df['Name']
dff = df.merge(df_count_merged, left_on='mergeonname', right_on='mergeonname')

In [50]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 247 entries, 0 to 246
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Rat                     247 non-null    object 
 1   Anrede 1_x              247 non-null    object 
 2   Vorname_x               247 non-null    object 
 3   Name_x                  247 non-null    object 
 4   Anrede 2_x              61 non-null     object 
 5   Firma (Postadresse)     56 non-null     object 
 6   Strasse (Postadresse)   81 non-null     object 
 7   Zusatz (Postadresse)    27 non-null     object 
 8   PLZ (Postadresse)       81 non-null     float64
 9   Ort (Postadresse)       81 non-null     object 
 10  Tel. (Postadresse)      38 non-null     object 
 11  Fax (Postadresse)       18 non-null     object 
 12  Homepage (Postadresse)  37 non-null     object 
 13  Firma (Wohnadresse)     5 non-null      object 
 14  Strasse (Wohnadresse)   180 non-null    ob

In [53]:
dff = dff[['Rat', 'Vorname_x', 'Name_x', 'Fraktion', 'Kanton', 'Anrede', 'Google Search Results']].copy()

In [54]:
dff

,Rat,Vorname_x,Name_x,Fraktion,Kanton,Anrede,Google Search Results
0,Nationalrat,Jean-Luc,Addor,V,VS,Conseiller national,"About 2,520 results (0.59 seconds)"
1,Nationalrat,Andreas,Aebi,V,BE,Nationalrat,"About 2,950 results (0.64 seconds)"
2,Nationalrat,Matthias,Aebischer,S,BE,Nationalrat,"About 7,020 results (0.58 seconds)"
3,Nationalrat,Thomas,Aeschi,V,ZG,Nationalrat,"About 25,500 results (0.51 seconds)"
4,Nationalrat,Céline,Amaudruz,V,GE,Conseillère nationale,"About 1,350 results (0.41 seconds)"
...,...,...,...,...,...,...,...
242,Ständerat,Hans,Wicki,RL,NW,Ständerat,"About 5,500 results (0.50 seconds)"
243,Ständerat,Benedikt,Würth,M-CEB,SG,Ständerat,"About 1,590 results (0.48 seconds)"
244,Ständerat,Roberto,Zanetti,S,SO,Ständerat,"About 9,160 results (0.42 seconds)"
245,Ständerat,Heidi,Z'graggen,M-CEB,UR,Ständerätin,About 135 results (0.35 seconds)


In [61]:
def clean(elem):
    try: 
        elem = elem.split(" ")[1]
        elem= int(elem.replace(",", ""))
        return elem
    except:
        return "N/A"

In [64]:
dff["Google Search Results"] = dff["Google Search Results"].apply(clean)

In [69]:
df_n = dff[dff["Rat"]=="Nationalrat"]

In [73]:
df_n.sort_values(by='Google Search Results', ascending=False).head(10)

,Rat,Vorname_x,Name_x,Fraktion,Kanton,Anrede,Google Search Results
99,Nationalrat,Philipp,Kutter,M-CEB,ZH,Nationalrat,377000
98,Nationalrat,Roger,Köppel,V,ZH,Nationalrat,102000
29,Nationalrat,Martin,Candinas,M-CEB,GR,Nationalrat,60500
66,Nationalrat,Andreas,Glarner,V,AG,Nationalrat,51700
6,Nationalrat,Sibel,Arslan,G,BS,Nationalrätin,39300
3,Nationalrat,Thomas,Aeschi,V,ZG,Nationalrat,25500
9,Nationalrat,Jacqueline,Badran,S,ZH,Nationalrätin,15600
82,Nationalrat,Alfred,Heer,V,ZH,Nationalrat,13700
114,Nationalrat,Thomas,Matter,V,ZH,Nationalrat,12500
55,Nationalrat,Kurt,Fluri,RL,SO,Nationalrat,11200


In [71]:
df_n.groupby('Fraktion')['Google Search Results'].mean()

Fraktion
G         2944.900000
GL        1730.250000
M-CEB    16434.548387
RL        3079.103448
S         3026.205128
V         6916.446429
Name: Google Search Results, dtype: float64

In [72]:
df_n.groupby('Kanton')['Google Search Results'].mean()

Kanton
AG     6759.812500
AI      499.000000
AR     1410.000000
BE     2954.291667
BL     4255.714286
BS     9955.200000
FR     1315.857143
GE      717.500000
GL     4110.000000
GR    14368.400000
JU      649.000000
LU     3302.111111
NE      256.000000
NW     7050.000000
OW     1020.000000
SG     4106.333333
SH     5880.000000
SO     6176.666667
SZ     3682.500000
TG     3318.714286
TI      992.375000
UR      753.000000
VD     1204.631579
VS     1710.875000
ZG    10976.666667
ZH    17470.457143
Name: Google Search Results, dtype: float64